In [8]:
!pip install transformers
!pip install razdel
!pip3 install -r "/content/drive/MyDrive/Colab Notebooks/dialogue_simplification/requirements.txt"
!pip install rnnmorph

     |████████████████████████████████| 2.1MB 5.8MB/s 
     |████████████████████████████████| 3.2MB 20.3MB/s 
     |████████████████████████████████| 890kB 26.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=ebda1dcc30b65fe9b205f39bd07674f4a21eb4d9f92822347eb9f57c50c2c6a7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 51kB 2.6MB/s 
     |████████████████████████████████| 194kB 6.9MB/s 
     |████████████████████████████████| 245kB 29.1MB/s 
     |████████████████████████████████| 112kB 36.2MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=c74f6017b38faf251a4f36a1866477ea970af7afa415acfb13ee67fd2262d82e
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval
     |█████████████████████

## Loading data

In [9]:
import pandas as pd
import os
from tqdm import tqdm_notebook
from razdel import tokenize
from functools import lru_cache
from rnnmorph.predictor import RNNMorphPredictor

PATH = '/content/drive/MyDrive/Colab Notebooks/dialogue_simplification'

In [10]:
# Read the data
df = pd.read_csv(os.path.join(PATH,'dev_sents.csv'))
pd.set_option('display.max_colwidth', 150)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3406 entries, 0 to 3405
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     3406 non-null   int64 
 1   INPUT:source   3406 non-null   object
 2   OUTPUT:output  3406 non-null   object
dtypes: int64(1), object(2)
memory usage: 80.0+ KB


In [ ]:
df.head()

,Unnamed: 0,INPUT:source,OUTPUT:output
0,3,"14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, после чего поселковый совет стал называться Советским.",14 декабря 1944 года рабочий посёлок Ички переименован в Советский.
1,4,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,"В 1960 году вышла модель 172А. Отличие в хвостовом оперении, в руле направления и в креплении шасси"
2,5,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,В выпущенной в 1960 году модель имела изменения в хвостовом оперении и руле направления
3,6,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,Изменения: в хвосте и руле направления с обратной стреловидностью и крепления для поплавкового шасси.\r\n
4,7,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,"Модель 172А с другим хвостовым оперением, креплениями для поплавкового шасси рулём направления с обратной стреловидностью появилась в 1960 году."


## Text preprocess

Группируем данные по INPUT:





In [ ]:
df_grouped = df.groupby('INPUT:source' ).agg({'OUTPUT:output': '(SEP)'.join})
input_corpus = df_grouped.index.values
output_corpus = [output_text.split('(SEP)') for output_text in df_grouped['OUTPUT:output'].values]
df_grouped['OUTPUT:output'] = output_corpus

In [ ]:
df_grouped.shape

((1000,), 1000)

['14',
 'декабрь',
 '1944',
 'год',
 'рабочий',
 'посёлок',
 'ичка',
 'быть',
 'переименованный',
 'в',
 'рабочий',
 'посёлок',
 'советский',
 ',',
 'посол',
 'что',
 'поселковый',
 'совет',
 'стать',
 'называться',
 'советский',
 '.']

In [ ]:
#@lru_cache(maxsize=)
def text_matching(corpus):
    predictor = RNNMorphPredictor(language="ru")
    new_corpus = []
    for sentence in tqdm_notebook(corpus, desc= 'preprocessing...'):
        sentence = [predictor.predict([token.text])[0].normal_form if token.text.isnumeric()==False else token.text for token in list(tokenize(sentence))]
        new_corpus.append(sentence)
    return new_corpus


In [ ]:
def lemmatizer(input_corpus, output_corpus):
    input_text_lemmatized = text_matching(input_corpus)
    # returns list of lemmatized group of text which corresponds to one input text
    output_text_lemmatized = [text_matching(group) for group in output_corpus]

    df_lemmatized = pd.DataFrame({'input': input_text_lemmatized, 'output': output_text_lemmatized})
    return df_lemmatized

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """



CPU times: user 14min 38s, sys: 17 s, total: 14min 55s
Wall time: 14min 29s


In [12]:
try:
    df_lemmatized
except NameError :
    df_lemmatized = pd.read_csv(os.path.join(PATH,'lemmatized_text.csv'))

In [14]:
df_lemmatized.head()

,Unnamed: 0,input,output
0,0,"['1960', 'год', 'быть', 'выпущенный', 'модель', '172', 'a', '.', 'изменение', ':', 'хвостовой', 'оперение', 'и', 'руль', 'направление', 'с', 'обра...","[['в', '1960', 'год', 'выйти', 'модель', '172', 'а', '.', 'отличие', 'в', 'хвостовой', 'оперение', ',', 'в', 'руль', 'направление', 'и', 'в', 'кре..."
1,1,"['14', 'декабрь', '1944', 'год', 'рабочий', 'посёлок', 'ичка', 'быть', 'переименованный', 'в', 'рабочий', 'посёлок', 'советский', ',', 'посол', 'ч...","[['14', 'декабрь', '1944', 'год', 'рабочий', 'посёлок', 'ичка', 'переименованный', 'в', 'советский', '.']]"
2,2,"['26', 'август', '2014', 'год', 'болдок', 'принять', 'участие', 'в', 'памятный', 'матч', 'кубок', 'футбольный', 'лига', 'против', '«', 'манчестер'...","[['26', 'август', '2014', 'год', 'болдок', 'выиграть', 'футбольный', 'матч', 'кубок', 'лига', 'у', 'команда', 'луи', 'ван', 'гала', 'со', 'счёт', ..."
3,3,"['armory', '—', 'клиент', 'с', 'различный', 'функция', 'для', 'повышение', 'безопасность', ',', 'являться', 'надстройка', ',', 'работающий', 'пове...","[['armory', '-', 'программа', 'с', 'различный', 'функция', 'для', 'повышение', 'безопасность', '.', 'являться', 'дополнение', ',', 'который', 'раб..."
4,4,"['concert', 'for', 'diana', '(', 'рус', '.', 'концерт', 'для', 'диана', ')', '—', 'концерт', 'память', ',', 'организованный', 'в', 'честь', 'диана...","[['concert', 'for', 'diana', '-', 'это', 'концерт', ',', 'посвящённый', 'память', 'принцесса', 'диана', 'уэльский', '.'], ['в', 'честь', 'диана', ..."


## Matching

In [ ]:
!wget "https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py"
!wget "https://github.com/huggingface/transformers/blob/master/examples/token-classification/run_ner.py"

--2021-03-16 09:04:18--  https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 991 [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]     991  --.-KB/s    in 0s      

2021-03-16 09:04:18 (44.1 MB/s) - ‘preprocess.py’ saved [991/991]

--2021-03-16 09:04:18--  https://github.com/huggingface/transformers/blob/master/examples/token-classification/run_ner.py
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘run_ner.py’

run_ner.py              [  <=>               ] 262.24K   850KB

In [ ]:
!export MAX_LENGTH=128
!export BERT_MODEL=bert-base-multilingual-cased
!export OUTPUT_DIR=germeval-model
!export BATCH_SIZE=32
!export NUM_EPOCHS=3
!export SAVE_STEPS=750
!export SEED=1


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/dialogue_simplification


In [ ]:
!python run_ner.py \
  --model_name_or_path BERT_MODEL \
  --train_file train.csv \
  --validation_file valid.csv \
  --output_dir OUTPUT_DIR \
  --do_train \
  --do_eval

2021-03-16 11:00:21.281469: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
03/16/2021 11:00:22 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0distributed training: False, 16-bits training: False
03/16/2021 11:00:22 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=OUTPUT_DIR, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=EvaluationStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_steps=0, logging_dir=runs/Mar16_11-00-22_11e9638d87fe, logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=None, n